rust中，一切皆为表达式，一切为类型。never类型，在rust记号为!，指得是表达式在线程崩溃、continue、break情况下表达式的返回值。never类型被编译器等价于任何类型。

类型，从物理的角度讲，指的是一种内存占用量和占用方式，所以，trait和struct是一样的东西。那么类型检查检查得是内存占用量和占用方式？rust是静态类型语言，即在编译期进行类型检查的语言，在运行时进行类型检查的语言属于动态类型。

+ 静态编译时可以确定内存占用大小及方式的类型，在rust中称为sized类型。
+ 静态编译时无法确定内存信息的类型Dynamic Sized Type，在rust中称为unsize类型，必须包装(box)在引用类型内被使用，自身不能作为变量、函数参数、枚举体变量。引用是sized类型。比如str类型可能是堆分配的，用引用类型包装它，称作&str包装了str。
+ sized类型也可以用引用包装，如&[T;N]包装[T;N]。而且包装sized类型更简单占用空间更少，称之为瘦指针，而unsize类型的引用则称为胖指针。
+ 我：有了sized类型和引用类型进行确定的栈分配，就可以在静态编译时确定栈帧格式及大小了。
+ str是DST，理论上可以通过`let s:str = *(String.from("hello"))`获得，但对于绑定DST的行为编译器会直接报错
```
let str = "Hello Rust"
let ptr = str.as_ptr()
let len = str.len()
```

trait可以定义方法，也可以定义变量。

+ 继承，又称子类型多态Subtype polymorphism。泛型，又称参数化多态Parametric polymorphism。
+ rust的泛型/参数化多态，是静多态，也就是说，在编译时泛型结构体、函数会单态化为多个具体类型的结构体、函数。
+ 特定多态/ad-hoc多态，指的是同一种行为抽象，在不同上下文中会有相应不同的行为实现。这体现在程序中就是：某认证trait的某方法，上下文为小明的行为，就和上下文为小黄的行为不一样。
+ 我：多态都是可以单态化的。换句话说，多态是复用。

+ 泛型，是表示不定type的表示方法。这种不定导致了动态性。简单地说，不定type就不能定调用的方法。
+ 多态是效果，和单态一样，都是一种存在形式。可以认为多态性就是动态性。
+ 孤儿规则，指的是type认证trait，必须trait和type至少有一个在当前crate中定义。
+ 重叠规则，指的是type不能重复被选中认证同一个trait，这也就避免了覆盖掉原有实现的机会。
+ trait是一种标签，作用是给type做认证。泛型trait，是一种可定制化标签。泛型动态的范围仅限于方法参数的类型：

```
trait Add<RHS = Self>{
    type Output;
    fn add(self, rhs: RHS) -> Self::Output;
}


impl Add for u32 {
    type Output = u32;
    fn add(self, other: u32) -> u32 {
        self + other
    }
}
```

+ 也有泛型type。★★★★★单态是不含有泛型的类型。可以想象，在类型系统中，单态是最小类型单位。下面的语法是同时选中某泛型type中所有单态：

```
struct Point<T>{x: T, y: T}
impl<T> Point<T>{
    fn new(x: T, y: T) -> Self{
        Point{x,y}
    }
}
```

+ ★★★★★也可以同时选中多type：
```
impl <T : Page + PerPage>Paginate for T{}
```

+ type认证泛型trait时，可以有两种写法，指定关联类型、或指定泛型：

```
impl Add for u32{
    type Output = u32;
}
// 大致等价
impl Add<u32, u32> for u32{
    
}
```


泛型type`struct Foo<T>{}`等价于`struct Foo<T:Sized>{}`，默认有sized泛型限制，所以如果有需要，可以使用语法`struct Foo<T:?Sized>{}`解除限制

&trait如&Display，称之为trait object。trait本身是DST，需要包装在它的引用trait object中。运行时，当trait_object.method被调用，通过trait object查出type进行调用，这个过程称之为动态分发。

+ ★★★★★rust中Clone的语义是堆复制或者栈复制，Copy是Clone的子认证trait，被认证的type只允许栈复制并且在传参、返回、绑定时拥有自动Clone行为。
+ Clone、Copy不会牵涉到所有权的概念。
+ 一般所谓的值复制是栈复制。
+ Move类似于搬家，类型从一个变量搬到另一个变量，这个过程就是所有权的转移和独占。被Move认证的type，在传参、返回、绑定时拥有自动搬家行为。当然被Move认证的type也可以被Clone认证。与Move相比，借用Borrow就传统的多。

+ 隐式type转换，就是强制type转换、type coercion。
+ 普通引用的`*`解引用操作不需要定义，而智能指针的`*`解引用需要deref()方法来定义。
+ 引用在运行时传参、调用自身方法时会自动解引用，引用自动解引用是rust中唯一的强转：
+ 就Box、Rc而言，引用自动解引用让Box、Rc变得透明就像不存在。
比如String类型实现了Deref<Target=str>：

```
let a = "hello".to_string();
let b = "world".to_string();
let c = a + &b;

```
比如，Vec<T>实现了Deref<Target=[T]>：
    
```
fn foo(s: &[i32]){}
let v = vec![1,2]    
foo(&v)
```

使用NewType模式的作用之一，是可以将远程类型包装为本地类型。反之，如果将本地类型放入Box、Option中，可以将本地类型转换成远程类型

计算机的一个字就是64位二进制数，64位又称字长。像u8,u16,u32都不满一个字。

&T和*const T，以及，&mut T和*mut T之间可以随意转换，唯一的区别是引用受编译器安全规则检查，而裸指针/原生指针则不。

static常量，生命周期为程序全局，存储在静态存储区。static常量不支持任何表达式，比如字符串字面量。
const常量，没有固定内存地址，是在编译时内联到每个使用它的位置，等价于生命周期为程序全局。const常量只支持 CTFE 的表达式。

+ 堆资源在程序中以Vec,String,File这些引用类型的形式存在。RAII机制保持一个堆资源只有唯一拥有析构权利的一个引用类型实例与之对应，通过引用类型在离开作用域时自动回收堆资源的方式达到避免遗留无用堆资源的效果。所以，RAII也有一个别名，叫做作用域界定的资源管理，SBRM。
+ 回收堆资源简称析构。利用花括号和Move特性可以提前析构：
```
{
    let mut v = vec![1,2,3];
    {
        v
    }
    // v.push(4);会报错
}

```

变量遮蔽shadowing，指的是重复let做变量绑定：
```
{
    let x = Vec![1]
    let x = Vec![2]
}

```
因为，不涉及定义域的变化，Vec![1]不会析构。

+ trait类型无法实例化、没有自己的实例，但rust中可以将trait认证的type实例同时认作trait实例，所以，type的实例即是type实例也是trait实例。并且由于trait是DST，trait实例trait_object记作&trait，如StrX认证的类型String也属于&strX。
+ 在编译期无法知道哪些type认证了某trait，可能有些type是unsize有些type是sized，这导致了trait是DST。如果trait被强行定义成如`trait Foo:Sized`，则一旦调用到unsize的trait对象时就会引发段错误。

From<T>和Into<U>是专门type转换方面的认证trait，目标type做了认证就不需要原type做了，具体而言，目标type通过了From认证，原type就通过了Into认证：
    
```
impl<T, U> Into<U> for T where U: From<T> 
```
一个type的From<T>认证多，说明实例化它的途径多：

```
let a = String::from("hello")
```

原类型&str通过了Into<String>，
另外在Into<U>认证中原类型是要搬家Move的

+ ★★★★★堆资源被多个引用共享是常态。所有权的本质是析构权。

+ 使用Rc<T>，就是使用垃圾回收机制(GC)，把析构权共享，一个堆资源对应的多个引用类型实例共享析构权。当共享析构权的所有Rc<T>离开作用域，析构堆资源。使用Rc<T>共享析构权有内存泄露的风险。

★★★★★枚举体就是标签联合体，标签用于显式的表明同一时刻那个成员在使用内存

&self是self:&self的缩写

+ Move是值移动
+ 结构体和枚举体属于值类型
+ 引用和Move的性能和值复制在一个数量级上。
+ 值就是类型，类型就是值。由于内存的堆和栈的设计，决定了编程语言中的“值”设计：值类型、引用类型。值类型经常被立即复制和立即回收，而引用类型通过析构权转移有更长的生命周期。

let绑定变量，指的是标识符与内存的绑定。作用域产生的绑定时效性就是绑定的生命周期。

绑定的析构顺序与声明顺序相反，类似先进后出的stack结构，所以最后声明的绑定生命周期最短：
```
let a = 1;
let b = 1;
let c = 1;
let d = 1;
```
d的时效性最短。
绑定和析构是反义词？

只读引用也被称为共享引用，因为可以多次只读引用。

我：借用/普通引用，没有析构权，是一个带有生命周期的值类型，跨作用域时是Copy行为。

借用规则：当绑定的右值是借用时，是否进行绑定。当右值的生命短于左值不予绑定。出借的右值，自己要命长。对于借用，要么把借出方的生命做长，要么把借用的时间做短：
```
fn f<'a>(x:&'a str,y:&'a str)->&'a str{
    if x.len() > y.len() { x } else { y }
}

fn main(){
    let x = String::from("Rust");
    let y = String::from("Yes");
    let res;
    let x_r = &x;
    {
        res = f(x_r,&y);
    }
    println!("{}",res)
}
```


这个例子中，出借方x,y命长于接收方res，所以可以绑定。

+ 标注生命周期是为了避免悬垂指针。
+ 不标注生命周期的类型方法，其返回类型的生命周期为self分配。
+ 原生类型的生命周期标注方式如&'a &str，自定义类型的生命周期标注方式如MyString<'a>

+ 对Box<T>使用操作符*进行解引用而转移所有权的行为，称之为解引用移动，Rc、Arc不支持。

+ Rc通过clone()方法增加的引用拥有析构权，称为强引用。downgrade()方法增加的引用没有析构权，称为弱引用。

+ Cell<T>、RefCell<T>是用于非mut type需要内部字段可修改的场景。Cell包装值类型或引用类型，RefCell包装引用类型。
+ 具体地，通过Cell的set()方法整个改变该字段，通过Cell的get()方法得到该字段的拷贝，通过Cell的get_mut()方法得到该字段的可变借用做局部改变。通过RefCell的borrow()和borrow_mut()方法得到只读借用、可变借用
+  Cell<T>、RefCell<T>最常配合引用使用如Rc<RefCell<T>>，表明这个字段是可变的且需要共享析构权。

高阶函数是以函数作为参数或返回。高阶函数底层基于函数指针。把函数指针指定做绑定时，必须显式指定类型fn()：
```
fn hello(){};
let f:fn() = hello
```
像fn(i32,i32)->i32，也是函数指针
如果不显式指定类型fn()，那么编译器会隐式指定类型fn(){hello}，fn(){hello}是实际的函数类型。

+ 每个函数每个闭包都是独立的type。
+ 闭包在rust中是一种trait认证，有三种闭包trait：Fn、FnMut、FnOnce，定义闭包的代码会被翻译成闭包trait认证的type。执行闭包，实际上是执行该结构体的call(())、call_once()、call_mut()方法。
+ 闭包的Move捕获方式，复制语义的捕获量克隆进闭包，移动语义的捕获量move进闭包，很像函数传参的缺省行为。

★★★★★Copy属于Clone，Clone和Copy都属于Move。

Fn、FnMut、FnOnce中只有FnMut有改变环境的能力。

```P189
let s = "hello".to_string();
let c = || s;
c()
```
捕获量移动语义且普通捕获：s被move进闭包，形成FnOnce认证的闭包，只能被使用一次

```P191
let s = "hello".to_string();
let c = move || {println!("{:?}", s)};
c()
c()
```
捕获量移动语义且move捕获：s被move进闭包，形成Fn认证的闭包，可反复使用

```P187
let s = "hello";
let c = || {println!("{:?}", s)};
c()
c()
println!("{:?}", s)
```
捕获量复制语义且普通捕获：s被借用进闭包，形成Fn认证的闭包，可反复使用，但借用会妨碍闭包作为函数返回值返回

```P190
let s = "hello";
let c = move || {println!("{:?}", s)};
c()
c()
println!("{:?}", s)
```
捕获量复制语义且move捕获：s被克隆进闭包，形成Fn认证的闭包，可反复使用，并且可以作为函数返回值

总结，捕获量复制语义使不使用move捕获，就是克隆和借用的区别。额外的优势是，捕获量复制语义且move捕获能让闭包结构体自身具有复制语义。

大部分情况下自动实现的是Fn，FnOnce只有在移动语义且不move的情况下才自动实现。所以，移动语义被捕获后就作废了，复制语义则可以继续使用。

FnMut:
```P192
let mut s = "rust".to_string();
{
    let mut c = || { s += " rust" };
    c();
    c();
    //println!("{:?}",s) error
}
println!("{:?}",s)
```

其实普通函数也通过了Fn认证。

impl Extend<char> for String{}，String类型可以通过extend方法，接受包含char的迭代器，完成extend自身的操作。在泛认证Extend中要求了：extend方法必须接受准迭代器类型，但对准迭代器所包含的元素类型采取泛化态度。

IntoIterator认证的into_iter()方法要求消耗自身。Iter的iter方法和IterMut的iter_mut方法分别返回不可变借用和可变借用

+ 迭代器是惰性的，只有当forof、any、fold、collect消费迭代器的next方法才会发生遍历行为。
+ forof循环中可以自动转换slice类型(&'a[T])为迭代器后遍历。
+ 迭代器的collect方法的本质是以迭代器为参数调用FromIterator的from_iter方法，collect的目标容器类型都实现了FromIterator trait。
+ 因为Iter类型的迭代器在forof时产生的循环变量为借用，所以闭包所处理的数据类型也是借用：
```
let arr = [1,2,3];
let result1 = arr.iter().any(|&x| x != 2);
let result2 = arr.iter().any(|x| *x != 2);

```


加法符操作接受借用，比较符操作不接受借用。

形如recevier.message形式的调用方式，称之为消息传递，点操作符左边的receiver是接收者，右边的部分被称为消息

形如..的更新语法，根据结构体本身的语义，决定是否转移所有权：
```P227
let book = Book {
    name: "编程之道",isbn: 20180404,version: 1
}
let book2 = Book{ version:2 ,..book}
```
如果Book是复制语义，就不转移book的所有权。如果Book是移动语义，则转移。

rust标准库std::default提供了Default trait，Default的作用：就像面向对象语言中的构造函数一样为结构体提供初始状态。大部分类型都通过了Default认证。

枚举体里的成员是值，而非类型。

如果要通过字符串的parse()方法，将字符串转成目标type，目标type必须通过FromStr认证。

字符编码描述的是字符和数字的关系。GB18300大于GBK大于GB2312。如果是unicode字符编码，那就是字符和码位的关系。unicode字符编码只规定了字符如何对应码位，却没有指定如何存储码位。UTF8、UTF16、UTF32规定了如何存储码位。其中UTF8是以8位为最小编码单元的不固定长编码方式，一个字符最小存储占用是8位。UTF16一个字符最小存储占用16位。以汉字“道”为例，它的码位是U+9053，对应二进制便是1001_0000_0101_0011，按UTF8编码生成字节序列1110_1001_10_000001_10_010011，相当于在原有二进制码位中间插入1110、10、10生成字节序列，这一过程同样也叫编码。

+ 在rust中，码位或utf8码元的数组可以通过from或from_utf8转换成str类型。
+ 在rust中使用u8类型表示一个utf8码元
+ &str是utf8字节序列。&str类型的reserve()方法、len()方法、capcity()方法返回的结果都是在描述码元，即都是在描述utf8字节序列
+ char类型与码位一一对应，即char的语义是单个unicode字符。char类型有escape_unicode()方法获取码位（十六进制）。
+ &str类型可以存储在任意地方：静态区，stack，heap。字面量在静态区。由String类型切片生成的&str是heap分配。由str::from_utf8生成的&str是stack分配

+ String类型是含有Vec<u8>类型成员变量的结构体。String类型的push和push_str是对vec<u8>追加u8即追加utf8码元。
+ String通过了Add认证，加法右操作符接受&str类型，如果传入String自动解引用为&str

+ array在rust中的类型记为[T,N]，Vector在rust中的类型记为Vec<T>，array是栈分配，Vector是堆分配。
+ 索引访问Vector越界的元素会报错，使用get()方法则不会。
+ Vector的truncate()clear(）方法不会释放内存，shrink_to_fit()会释放没使用的内存。

Vector的contains()、starts_with()、ends_with()方法、binary_search()方法只接受借用。

实现PartialEq需要实现eq。实现PartialOrd需要实现partial_cmp。PartialOrd和Ord的区别是，满足全序关系实现Ord的类型的每个实例之间要么Less、Equal、Greater，不可能存在只是不Equa但又不大不小的情况。浮点数、字符串满足偏序，使用partial_cmp()比较两者关系，返回Option<Ordering>。整数满足全序关系，使用cmp()比较两者关系，返回Ordering。

Vec<T>和[T,N]类型是[T]的子类型